### Import Necessary Packages

In [ ]:
import pandas as pd

### Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
track_metadata_df = pd.read_csv('/content/gdrive/MyDrive/Kaazima/EchonestDataset/song_data.csv')
count_play_df =pd.read_csv('/content/gdrive/MyDrive/Kaazima/EchonestDataset/10000.txt', sep='\t', header=None, names=['user','song','play_count'])

### Song_data.csv

In [ ]:
print('Number of rows:', track_metadata_df.shape[0])
print('Number of unique songs:', len(track_metadata_df.song_id.unique()))
display(track_metadata_df.head())
display(count_play_df.head())

Number of rows: 1000000
Number of unique songs: 999056


,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


Clearly, we have repeated track metadata.

### 10000.txt

In [ ]:
print('No of rows:', count_play_df.shape[0])
print('Number of unique users:', len(count_play_df.user.unique()))
display(count_play_df.head())

No of rows: 2000000
Number of unique users: 76353


,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


### Merge Datasets

Merge song_data.csv and 10000.txt based on the song_id.

In [ ]:
track_metadata_df.rename(columns = {'song_id': 'song'},inplace=True)
user_song_list_count = pd.merge(count_play_df, track_metadata_df.drop_duplicates(['song']), on='song', how ='left')
user_song_list_count.rename(columns={'play_count':'listen_count'},inplace=True)
display(user_song_list_count.head())

,user,song,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


### Popularity Recommendation Function

In [ ]:
def create_popularity_recommendation(train_data, user_id, item_id, n):
  
    #Get a count of user_ids for each unique song as recommendation score
    train_data_grouped = train_data.groupby([item_id]).agg({user_id: 'count'}).reset_index()
    train_data_grouped.rename(columns = {user_id: 'score'},inplace=True)
    
    #Sort the songs based upon recommendation score
    train_data_sort = train_data_grouped.sort_values(['score', item_id], ascending = [0,1])
    
    #Generate a recommendation rank based upon score
    train_data_sort['Rank'] = train_data_sort.score.rank(ascending=0, method='first')
        
    #Get the top n recommendations
    popularity_recommendations = train_data_sort.head(n)
    return popularity_recommendations

### Display Song Recommendations

In [ ]:
recommendations = create_popularity_recommendation(user_song_list_count,'user','title', 15)
display(recommendations)

,title,score,Rank
6836,Sehr kosmisch,8277,1.0
8725,Undo,7032,2.0
1964,Dog Days Are Over (Radio Edit),6949,3.0
9496,You're The One,6729,4.0
6498,Revelry,6145,5.0
6825,Secrets,5841,6.0
3437,Horn Concerto No. 4 in E flat K495: II. Romanc...,5385,7.0
2595,Fireflies,4795,8.0
3322,Hey_ Soul Sister,4758,9.0
8494,Tive Sim,4548,10.0


In [ ]:
recommendations.to_csv('/content/popular_songs.csv')

### Display Artist Recommendations

In [ ]:
display(create_popularity_recommendation(user_song_list_count,'user','artist_name', 15))

,artist_name,score,Rank
649,Coldplay,29422,1.0
2850,The Black Keys,19862,2.0
1651,Kings Of Leon,18747,3.0
1107,Florence + The Machine,18112,4.0
1370,Jack Johnson,17801,5.0
2946,The Killers,16063,6.0
2374,Radiohead,14890,7.0
736,Daft Punk,14715,8.0
2073,Muse,14005,9.0
1554,Justin Bieber,13959,10.0


In [ ]:
recommendations.to_csv('/content/popular_artists.csv')